In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 데이터로드

In [2]:
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')
data04 = pd.read_csv('202004.csv')

# 데이터 형식 변환(submission 과 동일한 형태)

In [3]:
df = data.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).AMT.sum().reset_index(drop=False)
df04 = data04.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).AMT.sum().reset_index(drop=False)

In [4]:
# 전체 데이터에 2004월 데이터 병합
df = pd.concat([df,df04],axis=0)
df

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
605,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
606,202004,충북,한식 음식점업,18911036160
607,202004,충북,호텔업,14121500
608,202004,충북,화장품 및 방향제 소매업,450507431


In [6]:
df07 = df.loc[df['REG_YYMM']==201907]
df1904 = df.loc[df['REG_YYMM']==201904]
df2004 = df04

In [16]:
# 19년 4월과 20년4월 중복 제거된 값을 제거된 값(코로나 영향)으로 생각하고 변수로 만듬 
covid = pd.concat([df1904.iloc[:,1:3],df2004.iloc[:,1:3]],axis=0).drop_duplicates(['CARD_SIDO_NM','STD_CLSS_NM'],keep=False).STD_CLSS_NM.unique()

In [8]:
# 동일하게 작년7월과 4월 데이터를 통해 중복제거된 업종 셀렉 
eval04 = pd.concat([df07.iloc[:,1:3],df04.iloc[:,1:3]],axis=0).drop_duplicates(['CARD_SIDO_NM','STD_CLSS_NM'],keep=False)

In [13]:
# 작년7월 제거된 값과 이번년4월을 중복제거 병합으로 새로운2007 업종 생성
test07 = pd.concat([df04.iloc[:,1:3],eval04],axis=0).drop_duplicates(['CARD_SIDO_NM','STD_CLSS_NM'],keep=False)

In [14]:
# feature의 순서 변경 및 submission 형태와 동일하게 맞추기위해 REG_YYMM 을 넣어줌
test07['REG_YYMM'] = 202007
test07= test07[['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']]
test07

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
0,202007,강원,건강보조식품 소매업
1,202007,강원,골프장 운영업
2,202007,강원,과실 및 채소 소매업
3,202007,강원,관광 민예품 및 선물용품 소매업
4,202007,강원,그외 기타 스포츠시설 운영업
...,...,...,...
4378,202007,충남,자동차 임대업
4380,202007,충남,정기 항공 운송업
4385,202007,충남,택시 운송업
4404,202007,충북,버스 운송업


In [17]:
# 아까 셀렉했었던 covid 업종들을 전체데이터와 test07 에 새로운 featuer를 만들어준다

df['covid'] = 1
for virus in covid:
    df.loc[(df['STD_CLSS_NM'] == virus),'covid'] = 0
    
test07['covid'] = 1
for virus in covid:
    test07.loc[(test04['STD_CLSS_NM'] == virus),'covid'] = 0

In [26]:
# df의 AMT를 제거한 피처개수와 test07의 피처개수가 동일한지 확인
len(df.columns)-1 == len(test07.columns)

True

In [27]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

dtypes = test07.dtypes
encoders = {}
for column in test07.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(test07[column])
        encoders[column] = encoder
        
test07_num = test07.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    test07_num[column] = encoder.transform(test07[column])

In [30]:
# df, test07 모두 라벨인코딩이 됬는지 확인
display(df_num)
display(test07_num)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,covid
0,201901,0,0,148354746,1
1,201901,0,1,1229200312,1
2,201901,0,2,1286979106,1
3,201901,0,3,30656770,0
4,201901,0,5,13101500,0
...,...,...,...,...,...
605,202004,16,36,1373635928,1
606,202004,16,37,18911036160,1
607,202004,16,38,14121500,1
608,202004,16,39,450507431,1


,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,covid
0,202007,0,0,1
1,202007,0,1,1
2,202007,0,2,1
3,202007,0,3,0
4,202007,0,5,0
...,...,...,...,...
4378,202007,15,28,0
4380,202007,15,30,0
4385,202007,15,35,0
4404,202007,16,15,0


In [32]:
test07_num

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,covid
0,202007,0,0,1
1,202007,0,1,1
2,202007,0,2,1
3,202007,0,3,0
4,202007,0,5,0
...,...,...,...,...
4378,202007,15,28,0
4380,202007,15,30,0
4385,202007,15,35,0
4404,202007,16,15,0


In [37]:
# 작년4월에서 올해 4월 증감율과 가감율을 계산해서 올해의 row 개수를 계산
df1904 = df.loc[df['REG_YYMM']==201904]
print((len(df04)-len(df1904))/len(df1904) * 100)
print((len(df07)-len(df1904))/len(df07)*100)
print(len(df07) * -3.6/100)
print(len(df07) * 0.9/100)
new_val = len(df07) + (-23+5)
new_val
# 새롭게 만든 639개 중에 증감율과 가감율을 통해 621개로 개수 확인

-3.6334913112164293
0.9389671361502347
-23.004
5.751


621

In [40]:
train_num = df_num.sample(frac=1, random_state=0)
X = train_num.drop(['AMT'], axis=1)
Y = np.log1p(train_num['AMT'])

In [44]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [43]:
def rmsle_1(y_pred, data):
    y_true = np.array(data.get_label())
    score= np.sqrt(np.square(np.log1p(y_pred + 1) - np.log1p(y_true + 1)).mean())
    return 'rmsle', score, False

def rmsle(y_pred, data): 
    y_true = np.array(data)
    score= np.sqrt(np.square(np.log1p(y_pred + 1) - np.log1p(y_true + 1)).mean())
    return print(score)

In [ ]:
model_xgb = xgb.XGBRegressor(max_depth=3,learning_rate=0.05,)